In [1]:
import os
import sys

sys.path.append("..")

import ast
import time
import pandas as pd
from dotenv import load_dotenv

from typing import Optional

from app.models.llms import GigaChatWrapper
# from app.utils.metric import validate
from app.utils.submit import generate_submit


AUTH_KEY = "Y2U4MDA4ZmUtNzdmOC00ODY5LWExYWMtMjI0NTY5MGRmNDAzOmYzMmVhOTc5LTVlNWUtNDIxZi1iZGRjLTZmNjY3ZTlhMjI1OQ=="
system_prompt = """Ты - профессиональный программист и ментор. Давай очень короткие ответы о синтаксических ошибках в коде. Не отправляй готовое решение или куски кода, только комментарии."""

giga = GigaChatWrapper(
    credentials=AUTH_KEY, 
    model="GigaChat-Pro", 
    system_prompt=system_prompt
)

/home/jovyan/.mlspace/envs/kuzhamuratov-temporalmovqgan/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading models...OK


In [2]:
test = pd.read_excel("../data/test/merged_executed.xlsx")

In [5]:
def predict(row: pd.Series) -> Optional[str]:
    time.sleep(1)
    prefix = f'''Описание задачи:
{row["description"]}\n
Решение студента:
{row["student_solution"]}'''
    
    if not pd.isna(row["test_result"]) and "error" in row["test_result"].lower():
        test_input = ast.literal_eval(row["test_input"].replace("nan", "None"))

        prefix = f'''{prefix}\n
Тест: {test_input[int(row["failed_test"])]}
Ошибка: {row["test_result"]}'''

    elif not pd.isna(row["test_result"]):
        test_input = ast.literal_eval(row["test_input"].replace("nan", "None"))
        test_output = ast.literal_eval(row["test_output"].replace("nan", "None"))

        prefix = f'''{prefix}\n
Тест: {test_input[int(row["failed_test"])]}
Ответ решения студента: {row["test_result"]}
Правильный ответ: {test_output[int(row["failed_test"])]}'''

    prefix += '''\n\nПримеры идеальных подсказок:
"Ошибка в открытых и скрытых тестах. Ваш код выводит некорректную переменную."
"Синтаксическая ошибка, попытка обратиться к ранее не заданной переменной. Проверьте, что Вы правильно указали все переменные, к которым обращаетесь в коде."
"Ошибка при возвращении данных функцией. Функция не должна использовать оператор print()."
"Ошибка при открытии файла. При обращении к файлу необходимо заключать его название в кавычки."
"Синтаксическая ошибка. При сравнении двух значений следует использовать знако двойного равенства."
Пиши ответ в похожем стиле.\nНачинай ответ с указания на ошибку, как в примерах. Ошибка в коде гарантированно есть. Нельзя отправлять куски кода.'''
    # print(prefix)
    try:
        answer = giga.ask(prefix)
    except:
        print("Модель отвалилась") 
        answer = "Ошибка в открытых и скрытых тестах. Ваш код выводит некорректную переменную."
        
    return answer

In [7]:
i = 3

print("Предикт GPT:")
print(predict(test.iloc[i]))
print("\n\n")
print("Решение студента:")
print(test["student_solution"][i])
print("\n\n")
print("Авторское решение:")
print(test["author_solution"][i])

Предикт GPT:
Ошибка в использовании переменной "discount" вместо "money".



Решение студента:
discount  = float(input())
money = int(input())

#ваш код ниже

print(f'Реализация проекта будет стоить {discount} тыс. руб. без скидки. Со скидой стоимость составит {money- (money * discount)} тыс. руб.')



Авторское решение:
discount  = float(input())
money = int(input())

#ваш код ниже

print(f'Реализация проекта будет стоить {money} тыс. руб. без скидки. Со скидой стоимость составит {money- (money * discount)} тыс. руб.')


In [8]:
generate_submit(
    test_solutions_path="../data/test/merged_executed.xlsx",
    predict_func=predict,
    save_path="../data/processed/gigachat_pro_prompt_v2_0.csv",
    use_tqdm=True,
)

Predicting: 100%|██████████| 325/325 [27:07<00:00,  5.01s/it] 
